<a href="https://colab.research.google.com/github/asdfasdf001234/2024-1-MLPRJ/blob/main/Resnet101.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import time
import shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.optim import lr_scheduler
import seaborn as sns

In [3]:
# pretrained 관련
import torch
import torchvision.transforms as v2
from torchvision import models

In [4]:
import torch
import numpy as np
import random

def set_random_seed(seed_value):
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)
    np.random.seed(seed_value)
    random.seed(seed_value)

# Set a random seed value
seed_value = 42
set_random_seed(seed_value)


# data 준비
- personal color data (train/val/test)
- data: 1-봄 웜 / 2-여름 쿨 / 3-가을 웜 / 4-겨울 쿨
- labeling: 0-봄 웜 / 1-여름 쿨 / 2-가을 웜 / 3-겨울 쿨

In [5]:
from torch.utils.data import Dataset , DataLoader
from torchvision import datasets
from torchvision.transforms import v2

In [6]:
from PIL import Image
from glob import glob

In [7]:
data_path = "./drive/MyDrive/Colab Notebooks/MLPRJ/Data"

In [ ]:
'''
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
'''

'\nfrom sklearn.model_selection import train_test_split\nX_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)\n'

In [8]:
import pandas as pd
import os
from glob import glob

def create_dataframe(data_path, label_list, data_type):
    df = pd.DataFrame({"path": [], "label": [], "class_id": []})
    img_list = glob(os.path.join(data_path, '*.jpg'))

    for img in img_list:
      file_name = os.path.splitext(os.path.basename(img))[0]
      label_index = int(file_name[0]) - 1
      if label_index == 0 or label_index == 2:   #웜톤
        label_index = 0                          #0으로 통일
      else:                                      #쿨톤
        label_index = 1                          #1으로 통일
      if 0 <= label_index < len(label_list):
        label = label_list[label_index]
        new_data = pd.DataFrame({"path": [img], "label": [label], "class_id": [label_index]})
        df = pd.concat([df, new_data], ignore_index=True)


    df[["path"]] = df[["path"]].astype(str)
    df[["label"]] = df[["label"]].astype(str)
    df[["class_id"]] = df[["class_id"]].astype(int)

    return df

'''
#웜쿨 분류 코드 (2분류 체계)
    for img in img_list:
      file_name = os.path.splitext(os.path.basename(img))[0]
      label_index = int(file_name[0]) - 1
      if label_index == 0 or label_index == 2:   #웜톤
        label_index = 0                          #0으로 통일
      else:                                      #쿨톤
        label_index = 1                          #1으로 통일
      if 0 <= label_index < len(label_list):
        label = label_list[label_index]
        new_data = pd.DataFrame({"path": [img], "label": [label], "class_id": [label_index]})
        df = pd.concat([df, new_data], ignore_index=True)
#수정 코드 끝
'''


'''
기존 코드 (4분류 체계)
    for img in img_list:
      file_name = os.path.splitext(os.path.basename(img))[0]
      label_index = int(file_name[0]) - 1  #진영님 코드와 동일
      if 0 <= label_index < len(label_list):
        label = label_list[label_index]
        new_data = pd.DataFrame({"path": [img], "label": [label], "class_id": [label_index]})
        df = pd.concat([df, new_data], ignore_index=True)
'''

'\n기존 코드 (4분류 체계)\n    for img in img_list:\n      file_name = os.path.splitext(os.path.basename(img))[0]\n      label_index = int(file_name[0]) - 1  #진영님 코드와 동일\n      if 0 <= label_index < len(label_list):\n        label = label_list[label_index]\n        new_data = pd.DataFrame({"path": [img], "label": [label], "class_id": [label_index]})\n        df = pd.concat([df, new_data], ignore_index=True)\n'

In [ ]:
# 기존 example 코드
train_path = data_path + '/train'
valid_path = data_path + '/val'
test_path = data_path + '/test'
label_list = ['spring', 'summer', 'autumn', 'winter']
#label_list = ['warm', 'cool']

train_df = create_dataframe(train_path, label_list, 'training')
val_df = create_dataframe(valid_path, label_list, 'validation')
test_df = create_dataframe(test_path, label_list, 'testing')

In [9]:
#2분류 코드
train_path = data_path + '/train'
valid_path = data_path + '/val'
test_path = data_path + '/test'
label_list = ['warm', 'cool']

train_df = create_dataframe(train_path, label_list, 'training')
val_df = create_dataframe(valid_path, label_list, 'validation')
test_df = create_dataframe(test_path, label_list, 'testing')

In [10]:
print(f"train_data: {len(train_df)}")
print(f"val_data:{len(val_df)}")
print(f"test_data:{len(test_df)}")

train_data: 413
val_data:137
test_data:137


In [11]:
#데이터 출력
print("Train DataFrame:")
print(train_df.head())
print("Validation DataFrame:")
print(val_df.head())
print("Test DataFrame:")
print(test_df.head())

Train DataFrame:
                                                path label  class_id
0  ./drive/MyDrive/Colab Notebooks/MLPRJ/Data/tra...  warm         0
1  ./drive/MyDrive/Colab Notebooks/MLPRJ/Data/tra...  warm         0
2  ./drive/MyDrive/Colab Notebooks/MLPRJ/Data/tra...  warm         0
3  ./drive/MyDrive/Colab Notebooks/MLPRJ/Data/tra...  warm         0
4  ./drive/MyDrive/Colab Notebooks/MLPRJ/Data/tra...  warm         0
Validation DataFrame:
                                                path label  class_id
0  ./drive/MyDrive/Colab Notebooks/MLPRJ/Data/val...  warm         0
1  ./drive/MyDrive/Colab Notebooks/MLPRJ/Data/val...  warm         0
2  ./drive/MyDrive/Colab Notebooks/MLPRJ/Data/val...  warm         0
3  ./drive/MyDrive/Colab Notebooks/MLPRJ/Data/val...  warm         0
4  ./drive/MyDrive/Colab Notebooks/MLPRJ/Data/val...  warm         0
Test DataFrame:
                                                path label  class_id
0  ./drive/MyDrive/Colab Notebooks/MLPRJ/Data/te

In [12]:
class BaseDataset(torch.utils.data.Dataset):
    def __init__(self , dataframe , transforms_):
        self.df = dataframe
        self.transforms_ = transforms_

    def __len__(self):
        return len(self.df)

    def __getitem__(self ,index):
        img_path = self.df.iloc[index]['path']
        img = Image.open(img_path).convert("RGB")
        transformed_img = self.transforms_(img)
        class_id = self.df.iloc[index]['class_id']
        return transformed_img , class_id

In [ ]:
'''
import cv2
def ConvertHSV(image):
  hsv_image =  cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
  cv2.imwrite('hsv_image.jpg', hsv_image)
'''

In [13]:
Transforms = v2.Compose([
    v2.RandomRotation(degrees=10),
    v2.RandomHorizontalFlip(p=0.8),
    #v2.ScaleJitter(target_size=(224,224)),
    v2.RandomAffine(degrees=45),
    #v2.ColorJitter(0.5, 0.5),
    #v2.RandomResizedCrop(size=(224, 224), antialias=True),

    v2.Resize((224,224)),
    v2.PILToTensor(),
    v2.ToDtype(torch.float32),
    v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

In [14]:
BATCH_SIZE = 22
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [15]:
#get dataloader

train_dataset = BaseDataset(train_df, Transforms) # train_transforms
val_dataset = BaseDataset(val_df, Transforms)
test_dataset = BaseDataset(test_df, Transforms)

In [16]:
'''
HSV로 바꾸려고 시도했던 코드
for image in train_dataset:
  print(image)
'''

'\nHSV로 바꾸려고 시도했던 코드\nfor image in train_dataset:\n  print(image)\n'

In [17]:
train_loader = DataLoader(train_dataset , batch_size=BATCH_SIZE , shuffle = True)
val_loader = DataLoader(val_dataset , batch_size=BATCH_SIZE)
test_loader = DataLoader(test_dataset , batch_size=BATCH_SIZE)

In [25]:
ResNet101 = models.resnet101(weights=models.ResNet101_Weights.DEFAULT)

Downloading: "https://download.pytorch.org/models/resnet101-cd907fc2.pth" to /root/.cache/torch/hub/checkpoints/resnet101-cd907fc2.pth
100%|██████████| 171M/171M [00:01<00:00, 162MB/s]


In [26]:
print(ResNet101)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [27]:
import torch.nn as nn
class MyResNet101(nn.Module):
    def __init__(self, pretrained_model):
        super(MyResNet101, self).__init__()
        self.backbone = pretrained_model

        #파라미터 고정(frozen)
        for param in self.backbone.parameters():
            param.requires_grad = True

        self.dropout = nn.Dropout(0.25)
        self.extra_layer = nn.Linear(1000, 2)
        #nn.Linear(1000, 4)



    def forward(self, x):
        x = self.backbone(x)
        x = self.dropout(x)
        x = self.extra_layer(x)
        #x = x.squeeze(1)
        return x


In [28]:
myresnet101 = MyResNet101(ResNet101)

In [29]:
myresnet101.cuda()

MyResNet101(
  (backbone): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
     

In [30]:
seed_value = 42
set_random_seed(seed_value)

EPOCHS = 100
logs = {"train_loss":[] , "train_acc":[] , "val_loss":[] , "val_acc":[]}

if os.path.exists('checkpoints') == False:
    os.mkdir('checkpoints')

criterion = nn.CrossEntropyLoss()


optimizer = optim.Adam(ResNet101.parameters(), lr=0.01)
Cosine_lr_scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)
#StepLR(optimizer, step_size=30, gamma=0.1)
#CosineAnnealingLR(optimizer, T_max=100)


patience = 10
counter = 0
best_loss = np.inf

In [31]:
def train(dataloader , model , loss_fn , optimizer , lr_scheduler):
    size = 0
    num_batches = len(dataloader)

    model.train()
    epoch_loss , epoch_correct = 0 , 0

    for i ,(data_ , target_) in enumerate(dataloader):

        data = data_.to(device)
        target = target_.to(device)

        pred = model(data)
        _, pred_max = torch.max(pred, 1)

        loss = loss_fn(pred, target)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        epoch_correct += (pred_max == target).type(torch.float).sum().item()
        size += target.size(0)


    train_acc = epoch_correct/size
    lr_scheduler.step()

    return train_acc , epoch_loss / num_batches


def test(dataloader , model , loss_fn):
    size = 0
    num_baches = len(dataloader)
    epoch_loss , epoch_correct= 0 ,0
    with torch.no_grad(): # grad 연산 X
        model.eval() # evaluation dropout 연산시
        for i, (data_ , target_) in enumerate(dataloader):

            data = data_.to(device)
            target = target_.to(device)

            pred = model(data)
            _, pred_max = torch.max(pred, 1)

            loss = loss_fn(pred, target)

            epoch_loss += loss.item()
            epoch_correct += (pred_max == target).type(torch.float).sum().item()
            size += target.size(0)

    test_acc = epoch_correct/size

    return test_acc  , epoch_loss / num_baches

In [32]:
for epoch in tqdm(range(EPOCHS)):
    train_acc , train_loss = train(train_loader ,
                                   myresnet101 ,
                                   criterion ,
                                   optimizer ,
                                   Cosine_lr_scheduler)

    val_acc , val_loss = test(val_loader , myresnet101 , criterion)
    print(f'epoch:{epoch} \
    train_loss = {train_loss:.4f} , train_acc:{train_acc:.4f} \
    val_loss = {val_loss:.4f} , val_acc:{val_acc:.4f} \
    learning rate: {optimizer.param_groups[0]["lr"]}')
    logs['train_loss'].append(train_loss)
    logs['train_acc'].append(train_acc)
    logs['val_loss'].append(val_loss)
    logs['val_acc'].append(val_acc)


    if val_loss < best_loss:
        counter = 0
        best_loss = val_loss
        #torch.save(myres50.state_dict() , "checkpoints/myvgg_16_best.pth")
    else:
        counter+=1

test_acc, val_loss = test(test_loader, myresnet101, criterion)
print(test_acc)
'''
    if counter >= patience:
        test_acc , val_loss = test(test_loader , myresnet101 , criterion)
        print("Early stop !")
        print(test_acc)
        break
'''


  1%|          | 1/100 [00:11<18:11, 11.03s/it]

epoch:0     train_loss = 3.6951 , train_acc:0.5932     val_loss = 80929377.1429 , val_acc:0.3650     learning rate: 0.009000000000000001


  2%|▏         | 2/100 [00:22<17:58, 11.01s/it]

epoch:1     train_loss = 1.7770 , train_acc:0.5981     val_loss = 1741.6013 , val_acc:0.6496     learning rate: 0.008100000000000001


  3%|▎         | 3/100 [00:33<17:54, 11.08s/it]

...

epoch:14     train_loss = 0.7448 , train_acc:0.6344     val_loss = 0.7467 , val_acc:0.6204     learning rate: 0.0020589113209464917


 16%|█▌        | 16/100 [02:53<15:06, 10.79s/it]

epoch:15     train_loss = 0.6468 , train_acc:0.6392     val_loss = 0.6537 , val_acc:0.6204     learning rate: 0.0018530201888518425
epoch:16     train_loss = 0.7047 , train_acc:0.6320     val_loss = 1.4405 , val_acc:0.6350     learning rate: 0.0016677181699666583


 16%|█▌        | 16/100 [03:05<16:13, 11.59s/it]

Early stop !
0.635036496350365
